In [1]:
from subpred.protein_go_datasets import get_transmembrane_transporter_dataset, get_stats

## Full dataset

In [2]:
dataset_athaliana = get_transmembrane_transporter_dataset(  # athaliana
    organism_ids=[3702],
    swissprot_only=False,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=False,
    max_sequence_evidence_code=2,
    remove_proteins_without_gene_names=False,
)
get_stats(*dataset_athaliana)

cd-hit: clustered 1773 sequences into 646 clusters at threshold 50
cd-hit: clustered 1773 sequences into 923 clusters at threshold 70
cd-hit: clustered 1773 sequences into 1203 clusters at threshold 90
cd-hit: clustered 1773 sequences into 1554 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering                   
False              False         computational protein_level              100                      3   
                                                                          None                     3   
                                               transcript_level           50                       3   
                                                                          70                       4   
                                                                          90                       7   
                                                                          100                     37   
                                                                          None                    49   
                   True          computational protein_level              50                      87   
                                                                          70                     114   
                                                                          90                     153   
                                                                          100                    329   
                                                                          None                   433   
                                               transcript_level           50                       7   
                                                                          70                      13   
                                                                          90                      19   
                                                                          100                     84   
                                                                          None                   164   
                                 experiment    protein_level              50                      25   
                                                                          70                      33   
                                                                          90                      38   
                                                                          100                     41   
                                                                          None                    42   
                                               transcript_level           50                       3   
                                                                          70                       5   
                                                                          90                       5   
                                                                          100                      6   
                                                                          None                     6   
True               True          computational protein_level              50                     206   
                                                                          70                     287   
                                                                          90                     370   
                                                                          100                    408   
                                                                          None                   418   
                                               transcript_level           50                     208   
                                                                          70                     338   
                                                                          90                     462   
                                                                          100                    492   
                                                                          None                   504   
                              

## Filtered Subset

In [3]:
dataset_athaliana = get_transmembrane_transporter_dataset(  # athaliana
    organism_ids=[3702],
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=True,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_athaliana)

cd-hit: clustered 420 sequences into 276 clusters at threshold 50
cd-hit: clustered 420 sequences into 344 clusters at threshold 70
cd-hit: clustered 420 sequences into 403 clusters at threshold 90
cd-hit: clustered 420 sequences into 419 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering                   
True               True          experiment  protein_level              50                     276   
                                                                        70                     344   
                                                                        90                     403   
                                                                        100                    419   
                                                                        None                   420   

                                                                                    n_terms  
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering           
True               True          experiment  protein_level              50              264  
                                                                        70              281  
                                                                        90              288  
                                                                        100             288  
                                                                        None            288

## Sugar/Amino subset

In [ ]:
df_sequences_athaliana, df_goa_athaliana = dataset_athaliana
df_goa_athaliana = (
    df_goa_athaliana[
        df_goa_athaliana.go_term_ancestor.isin(
            [
                "sugar transmembrane transporter activity",
                "amino acid transmembrane transporter activity",
            ]
        )
    ][["Uniprot", "go_term_ancestor"]]
    .drop_duplicates()
    .reset_index(drop=True)
)  # .go_term_ancestor.value_counts()
df_sequences_athaliana = df_sequences_athaliana[
    df_sequences_athaliana.index.isin(df_goa_athaliana.Uniprot)
]
print("before clustering:")
df_goa_athaliana.go_term_ancestor.value_counts()

go_term_ancestor
amino acid transmembrane transporter activity    33
sugar transmembrane transporter activity         32
Name: count, dtype: int64

Clustering at 70%

In [ ]:
from subpred.cdhit import cd_hit

cluster_representatives_70 = set(
    cd_hit(df_sequences_athaliana.sequence, identity_threshold=70)
)
df_sequences_athaliana = df_sequences_athaliana[
    df_sequences_athaliana.index.isin(cluster_representatives_70)
]
df_goa_athaliana = df_goa_athaliana[
    df_goa_athaliana.Uniprot.isin(cluster_representatives_70)
]
df_goa_athaliana.go_term_ancestor.value_counts()

dataset_athaliana = (df_sequences_athaliana, df_goa_athaliana)


cd-hit: clustered 65 sequences into 54 clusters at threshold 70


go_term_ancestor
sugar transmembrane transporter activity         28
amino acid transmembrane transporter activity    26
Name: count, dtype: int64

In [ ]:
# package them back to avoid confusion

## Feature generation

In [ ]:
from subpred.features import calculate_features
from subpred.pssm import calculate_pssm_feature
from subpred.compositions import calculate_comp, ALPHABET_3DI, AMINO_ACIDS
from subpred.structural_sequences import get_3Di_sequences
from subpred.embeddings import get_nlp_features
from sklearn.preprocessing import scale
import pandas as pd
import numpy as np

# Can take a long time if cache is empty
df_sequences, df_uniprot_goa = dataset_athaliana
series_sequences = df_sequences.sequence
series_accessions = df_sequences.index

# 3Di sequence features
sequences_3Di = get_3Di_sequences(series_accessions)
# Are there as many 3Di sequences as AA sequences? If yes, maybe take intersection
assert len(sequences_3Di) == len(series_sequences)

# original sequences features
df_aac = calculate_comp(series_sequences, k=1, alphabet=AMINO_ACIDS)
df_paac = calculate_comp(series_sequences, k=2, alphabet=AMINO_ACIDS)
df_kmer3 = calculate_comp(series_sequences, k=3, alphabet=AMINO_ACIDS)

pssm_folder = "../data/datasets/pssm/"
blastdb_folder = "../data/datasets/blastdb/"
verbosity_pssm = 1  # only print if no pssm found
df_pssm_50_1 = calculate_pssm_feature(
    series_sequences,
    tmp_folder=pssm_folder + "pssm_uniref50_1it",
    blast_db=blastdb_folder + "uniref50/uniref50.fasta",
    iterations=1,
    psiblast_threads=-1,
    verbosity=verbosity_pssm,
    feature_name="PSSM_50_1",
)
df_pssm_50_3 = calculate_pssm_feature(
    series_sequences,
    tmp_folder=pssm_folder + "pssm_uniref50_3it",
    blast_db=blastdb_folder + "uniref50/uniref50.fasta",
    iterations=3,
    psiblast_threads=-1,
    verbosity=verbosity_pssm,
    feature_name="PSSM_50_3",
)
df_pssm_90_1 = calculate_pssm_feature(
    series_sequences,
    tmp_folder=pssm_folder + "pssm_uniref90_3it",
    blast_db=blastdb_folder + "uniref90/uniref90.fasta",
    iterations=1,
    psiblast_threads=-1,
    verbosity=verbosity_pssm,
    feature_name="PSSM_90_1",
)
df_pssm_90_3 = calculate_pssm_feature(
    series_sequences,
    tmp_folder=pssm_folder + "pssm_uniref90_3it",
    blast_db=blastdb_folder + "uniref90/uniref90.fasta",
    iterations=3,
    psiblast_threads=-1,
    verbosity=verbosity_pssm,
    feature_name="PSSM_90_3",
)
df_pssm_meta = pd.concat(
    [df_pssm_50_1, df_pssm_50_3, df_pssm_90_1, df_pssm_90_3], axis=1
)

df_3Di_AAC = calculate_comp(sequences=sequences_3Di, k=1, alphabet=ALPHABET_3DI)
df_3Di_PAAC = calculate_comp(sequences=sequences_3Di, k=2, alphabet=ALPHABET_3DI)
df_3Di_KMER3 = calculate_comp(sequences=sequences_3Di, k=3, alphabet=ALPHABET_3DI)

# AA Embeddings
df_embeddings_prott5_AA = get_nlp_features(
    sequences=series_sequences,
    model="protT5",
    sequence_type="AA",
    half_precision=True,
)
df_embeddings_prostt5_AA = get_nlp_features(
    sequences=series_sequences,
    model="prostT5",
    sequence_type="AA",
    half_precision=True,
)
# 3Di Embeddings
df_embeddings_prott5_3Di = get_nlp_features(
    sequences=sequences_3Di,
    model="prostT5",
    sequence_type="3Di",
    half_precision=True,
)

np.random.seed(0)
df_dummy_feature = pd.DataFrame(
    np.random.rand(54, 1024),
    index=df_embeddings_prott5_AA.index,
    columns=[f"dummy{i}" for i in range(1024)],
)

features_list = [
    ("AAC_AA", df_aac),
    ("PAAC_AA", df_paac),
    ("KMER3_AA", df_kmer3),
    ("PSSM_50_1", df_pssm_50_1),
    ("PSSM_50_3", df_pssm_50_3),
    ("PSSM_90_1", df_pssm_90_1),
    ("PSSM_90_3", df_pssm_90_3),
    ("PSSM_META", df_pssm_meta),
    ("AAC_3Di", df_3Di_AAC),
    ("PAAC_3Di", df_3Di_PAAC),
    ("KMER3_3Di", df_3Di_KMER3),
    ("EMBEDDINGS_PROTT5_AA", df_embeddings_prott5_AA),
    ("EMBEDDINGS_PROSTT5_AA", df_embeddings_prostt5_AA),
    ("EMBEDDINGS_PROTT5_3DI", df_embeddings_prott5_3Di),
    ("DUMMY", df_dummy_feature),
]

series_labels = (
    df_uniprot_goa[df_uniprot_goa.Uniprot.isin(series_accessions)]
    .set_index("Uniprot")
    .go_term_ancestor
)

For each Feature: Outlier detection, numpy conversion 

In [30]:
from sklearn.preprocessing import LabelEncoder


def get_ml_dataset(df_features, df_goa):
    sample_names = df_features.index.to_numpy()
    feature_names = df_features.columns.to_numpy()
    series_labels = df_goa[df_goa.Uniprot.isin(sample_names)].set_index("Uniprot")

    assert not series_labels.index.duplicated().any()

    label_encoder = LabelEncoder()

    X = df_features.loc[sample_names].to_numpy()
    y_str = series_labels.loc[sample_names].to_numpy().ravel()
    y = label_encoder.fit_transform(y_str)

    return X, y, sample_names, feature_names, label_encoder

In [51]:
# X,y,sample_names,feature_names = get_ml_dataset(df_features_std)
X, y, sample_names, feature_names, label_encoder = get_ml_dataset(df_3Di_PAAC, df_goa_athaliana)

In [52]:
ml_datasets = [(feature_name, *get_ml_dataset(df_features=df_features, df_goa=df_goa_athaliana)) for feature_name, df_features in features_list]
ml_datasets

[('AAC_AA',
  array([[0.06639004, 0.00414938, 0.02074689, ..., 0.10373444, 0.01659751,
          0.05394191],
         [0.07708333, 0.02083333, 0.03333333, ..., 0.11666667, 0.02291667,
          0.0375    ],
         [0.07287449, 0.0242915 , 0.02834008, ..., 0.08502024, 0.00809717,
          0.03238866],
         ...,
         [0.05818182, 0.01818182, 0.03818182, ..., 0.07454545, 0.01090909,
          0.04909091],
         [0.08243728, 0.01075269, 0.02867384, ..., 0.07526882, 0.0125448 ,
          0.03405018],
         [0.0867052 , 0.01541426, 0.00770713, ..., 0.06358382, 0.0327553 ,
          0.03468208]], shape=(54, 20)),
  array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
         1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
         0, 0, 0, 1, 0, 1, 0, 0, 1, 0]),
  array(['Q84WN3', 'Q8GUM3', 'Q8L9J7', 'Q8VZ80', 'Q9FMF7', 'Q9SMM5',
         'Q94KE0', 'Q9C8E7', 'Q9FG00', 'Q9FKS8', 'Q9FY94', 'O81845',
         'Q8GX78', 'Q8LFH5', 'Q9CA93', 

In [53]:
# # TODO check each feature dataset for outliers, is there a sample that is found in multiple?

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

from sklearn.ensemble import IsolationForest

outlier_detector = make_pipeline(
    StandardScaler(),
    PCA(n_components=0.95),
    IsolationForest(contamination="auto", random_state=0),
)
outliers = outlier_detector.fit_predict(X)  # -1 for outliers, 1 for inliers
is_outlier = outliers == -1
print(is_outlier.sum(), "outliers found")


remove_outliers = False
if remove_outliers:
    X = X[outliers != -1]
    y = y[outliers != -1]

1 outliers found


Plots? If outliers found: show in plot!

In [54]:
import seaborn as sns

# sns.clustermap(X, row_colors=["yellow" if label == 1 else "blue" for label in y])

## List of models

In [55]:
# TODO linear svm, linear svm with nyström, DNN

## Model evaluation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    RepeatedStratifiedKFold,
    StratifiedKFold,
    LeaveOneOut,
)
from sklearn.metrics import f1_score, balanced_accuracy_score


model = make_pipeline(VarianceThreshold(), StandardScaler(), SelectKBest(), SVC())

# TODO test different values!!!
max_features = int(np.sqrt(len(feature_names)))
max_features = int(len(sample_names))

param_grid = {
    "selectkbest__k": list(range(1, max_features, 1)),
    "svc__class_weight": ["balanced"],
    # "svc__C":[0.1, 1, 10],
    # "svc__gamma": ["scale", "auto", 0.1, 1, 10]
}

gridsearch = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=StratifiedKFold(5),
    n_jobs=-1,
)

# Testing Gridsearch on the whole dataset first (unreliable)
gridsearch_results = gridsearch.fit(X, y)
print(gridsearch_results.best_score_)
print(gridsearch_results.best_params_)
df_results = pd.DataFrame(gridsearch_results.cv_results_)
display(
    df_results.filter(like="test_score", axis=1).sort_values("rank_test_score").head()
)

# Nested loop (sound results):
gridsearch.n_jobs = 1
nested_crossval_results = cross_val_score(
    gridsearch,
    X,
    y,
    cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=0),
    scoring="balanced_accuracy",
    n_jobs=-1,
)
nested_crossval_results.mean(), nested_crossval_results.std()

0.9833333333333334
{'selectkbest__k': 51, 'svc__class_weight': 'balanced'}


,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
52,1.000000,1.0,0.916667,1.0,1.0,0.983333,0.033333,1
51,1.000000,1.0,0.916667,1.0,1.0,0.983333,0.033333,1
50,1.000000,1.0,0.916667,1.0,1.0,0.983333,0.033333,1
49,0.916667,1.0,0.916667,1.0,1.0,0.966667,0.040825,4
47,0.816667,1.0,1.000000,1.0,1.0,0.963333,0.073333,5


(np.float64(0.9166666666666667), np.float64(0.0787400787401181))

In [ ]:
# TODO violin plots or boxplots, with long-form data in seaborn